In [18]:
!pip install streamlit
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import streamlit as st
import cv2
from zipfile import ZipFile
from tensorflow import keras
from tensorflow.keras.layers import Rescaling
from sklearn.model_selection import train_test_split
import matplotlib.image as mimg
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from tensorflow.keras.utils import image_dataset_from_directory
!pip install opendatasets
import opendatasets

In [19]:
opendatasets.download("https://www.kaggle.com/datasets/harbhajansingh21/german-traffic-sign-dataset")

Skipping, found downloaded files in "./german-traffic-sign-dataset" (use force=True to force download)


In [20]:
# The dataset is split into a folder containing the images divided into subdirectories by class, and
# a csv file containing the labels for each subdirectory.
label_df = pd.read_csv('/content/german-traffic-sign-dataset/signname.csv')
for i in range(len(label_df)):
  print(f'{i}: {label_df.iloc[i][1]}')

0: Speed limit (20km/h)
1: Speed limit (30km/h)
2: Speed limit (50km/h)
3: Speed limit (60km/h)
4: Speed limit (70km/h)
5: Speed limit (80km/h)
6: End of speed limit (80km/h)
7: Speed limit (100km/h)
8: Speed limit (120km/h)
9: No passing
10: No passing for vehicles over 3.5 metric tons
11: Right-of-way at the next intersection
12: Priority road
13: Yield
14: Stop
15: No vehicles
16: Vehicles over 3.5 metric tons prohibited
17: No entry
18: General caution
19: Dangerous curve to the left
20: Dangerous curve to the right
21: Double curve
22: Bumpy road
23: Slippery road
24: Road narrows on the right
25: Road work
26: Traffic signals
27: Pedestrians
28: Children crossing
29: Bicycles crossing
30: Beware of ice/snow
31: Wild animals crossing
32: End of all speed and passing limits
33: Turn right ahead
34: Turn left ahead
35: Ahead only
36: Go straight or right
37: Go straight or left
38: Keep right
39: Keep left
40: Roundabout mandatory
41: End of no passing
42: End of no passing by vehic

<ipython-input-20-05e4952b039c>:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f'{i}: {label_df.iloc[i][1]}')


In [21]:
import pickle

with open('/content/german-traffic-sign-dataset/train.p', 'rb') as file:
    train_data = pickle.load(file)

print(train_data.keys())

if isinstance(train_data, dict):
    train_images = train_data.get('features')
    train_labels = train_data.get('labels')

dict_keys(['coords', 'labels', 'features', 'sizes'])


In [22]:
print(train_labels)
print(label_df['SignName'][41])

[41 41 41 ... 25 25 25]
End of no passing


In [23]:
import pickle

with open('/content/german-traffic-sign-dataset/test.p', 'rb') as file:
    test_data = pickle.load(file)

if isinstance(test_data, dict):
    test_images = test_data.get('features')
    test_labels = test_data.get('labels')

In [24]:
train_images = np.array(train_images)
test_images = np.array(test_images)

In [25]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
image_generator = ImageDataGenerator(rescale = 1/255)
train_data = image_generator.flow(train_images, train_labels, batch_size=32)
test_data = image_generator.flow(test_images, test_labels, batch_size=32)

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(43, activation='softmax'),
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data,  epochs=10)

Epoch 1/10
   2/1088 ━━━━━━━━━━━━━━━━━━━━ 58s 54ms/step - accuracy: 0.0234 - loss: 3.7849

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1088/1088 ━━━━━━━━━━━━━━━━━━━━ 63s 56ms/step - accuracy: 0.3417 - loss: 2.4613
Epoch 2/10
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 58s 53ms/step - accuracy: 0.8516 - loss: 0.4846
Epoch 3/10
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 65s 60ms/step - accuracy: 0.9217 - loss: 0.2628
Epoch 4/10
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 75s 54ms/step - accuracy: 0.9448 - loss: 0.1834
Epoch 5/10
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 58s 54ms/step - accuracy: 0.9544 - loss: 0.1514
Epoch 6/10
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 82s 54ms/step - accuracy: 0.9637 - loss: 0.1159
Epoch 7/10
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 81s 53ms/step - accuracy: 0.9692 - loss: 0.0998
Epoch 8/10
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 56s 51ms/step - accuracy: 0.9744 - loss: 0.0841
Epoch 9/10
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 93s 62ms/step - accuracy: 0.9774 - loss: 0.0716
Epoch 10/10
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 58s 53ms/step - accuracy: 0.9797 - loss: 0.0666


In [ ]:
plt.plot(history.history["accuracy"], label = "training accuracy")
plt.legend()
plt.show()

In [28]:
model.evaluate(test_data)

  9/395 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.9249 - loss: 0.3435

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


395/395 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.9343 - loss: 0.3883


[0.3967820107936859, 0.9360253214836121]

In [53]:
y_pred = []
y_true = []

for i in range(len(test_data)):
  batch_x, batch_y = test_data[i]
  batch_pred = model.predict(batch_x)
  batch_pred_classes = np.argmax(batch_pred, axis=1)

  y_pred.extend(batch_pred_classes)
  y_true.extend(batch_y)

y_pred = np.array(y_pred)
y_true = np.array(y_true)
y_pred = label_df['SignName'][y_pred]
y_true = label_df['SignName'][y_true]
print(y_pred)
print(y_true)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━

In [48]:
print(y_pred)
print(y_true)

['Speed limit (80km/h)' 'Slippery road' 'Speed limit (50km/h)' ...
 'No entry' 'Speed limit (30km/h)' 'Right-of-way at the next intersection']
['Vehicles over 3.5 metric tons prohibited' 'Speed limit (30km/h)'
 'Keep right' ... 'End of speed limit (80km/h)' 'Speed limit (100km/h)'
 'No passing for vehicles over 3.5 metric tons']


In [54]:
from tensorflow.keras.models import save_model

save_model(model, 'model.keras')